In [1]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

spark = SparkSession.builder.appName('Leetcode').getOrCreate()

24/12/23 12:24:58 WARN Utils: Your hostname, madiv resolves to a loopback address: 127.0.1.1; using 192.168.1.11 instead (on interface wlo1)
24/12/23 12:24:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/23 12:24:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark_df = spark.read.option('delimiter',',').option('header',True).csv('Dataset.csv')
pandas_df = pd.read_csv('Dataset.csv')
spark_df.show()
pandas_df.head()

+----------+--------+----------+
|product_id|low_fats|recyclable|
+----------+--------+----------+
|         0|       Y|         N|
|         1|       Y|         Y|
|         2|       N|         Y|
|         3|       Y|         Y|
|         4|       N|         N|
+----------+--------+----------+



,product_id,low_fats,recyclable
0,0,Y,N
1,1,Y,Y
2,2,N,Y
3,3,Y,Y
4,4,N,N


### SQL Solution

In [3]:
def find_products_SQL(products: pyspark.sql.dataframe.DataFrame) -> pyspark.sql.dataframe.DataFrame:
    products.createOrReplaceTempView('Products')
    sql_query = \
    '''
       SELECT product_id FROM
       Products WHERE low_fats = 'Y' AND
       RECYCLABLE = 'Y'
    '''
    product_ids = spark.sql(sql_query)
    return product_ids

product_ids = find_products_SQL(spark_df)
product_ids.show()

+----------+
|product_id|
+----------+
|         1|
|         3|
+----------+



### Spark Solution

In [4]:
def find_products(products: pyspark.sql.dataframe.DataFrame) -> pyspark.sql.dataframe.DataFrame:
    product_ids = products\
                    .filter((products['low_fats'] == 'Y') & (products['recyclable'] == 'Y'))\
                    .select('product_id')
    return product_ids

product_ids = find_products(spark_df)
product_ids.show()

+----------+
|product_id|
+----------+
|         1|
|         3|
+----------+



### Pandas Solution

In [5]:
def find_products_pandas(products: pd.DataFrame) -> pd.DataFrame:
    product_ids = products[(products['low_fats'] == 'Y') & (products['recyclable'] == 'Y')].product_id
    product_ids = pd.DataFrame(product_ids)
    return product_ids

product_ids = find_products_pandas(pandas_df)
product_ids.head()

,product_id
1,1
3,3
